## 2 Number Guessing Game


In [5]:
import random
import math
import ipywidgets as widgets
from IPython.display import display, clear_output

# Global variables to track the number of guesses and the score
numq = 0  # Initialize the number of guesses
current_score = 0  # Initialize score to 0

# Function to generate two distinct secret numbers
def respond(n):
    lst = list(range(1, n + 1))
    n1 = random.sample(lst, 1)[0]
    lst.remove(n1)
    n2 = random.sample(lst, 1)[0]
    return n1, n2

# Function to test if numbers are in the given range
def test(n1, n2, end1, end2):
    lst = list(range(min(end1, end2), max(end1, end2) + 1))
    count = sum(num in lst for num in (n1, n2))
    return count

# Function to calculate the optimal score
def optimal(n):
    return math.ceil(math.log2(n - 1))

# Function to calculate the score
def score(numq, n, current_score, guessed_correctly):
    if guessed_correctly:
        # Only reward if the number of guesses is within optimal range
        if numq <= optimal(n):
            current_score += (optimal(n) - numq + 2) * 100
        elif numq == optimal(n)+1:
            current_score += 100
    return current_score

In the code chunks above, we have implemented different ideas from the MTH 496 Senior Thesis research project. We set up functions that chose distict and random values within an interval defined by the player. Moreover, we set up a function to test whether the two numbers are within an interval that is set by the player. We can think of the initial interval as a set $S$, from the respond function. Additionally, the test function tests to see how many numbers are in a set $Q$ which is a subset of $S$. Next, we implement the proven optimal strategy for this game as defined by the MTH 496 research. Here, we take a given set size and return $K(n)$ which defined the optimal number of guesses it should take for the Questioner to determine at least one of the Responder's secret numbers. This function was then used in our scoring. This scoring function took input parameters of the number of guesses the Questioner has, the initial set size $|S| = n$, the player's current score, and whether or not they guessed correctly. If they did guess correctly, then they will be rewarded points based on how efficiently they found one of the Responder's secret numbers. We elected to give them points only if they ask less than the optimal number of questions plus 1. Additionally, the player recieves more points if they answer faster than expected. For example, if they immediately guess a number and are correct, they will recieve more points than if they employ an optimal strategy to solve this game. However, this is risky as they will recieve 0 points if they guess incorrectly. 

In [6]:
# Function to run the game
def run_game():
    global n1, n2, current_score, numq, saved_high_value
    n = high_input.value  # Get the user-defined upper limit
    if n < 2:  # Ensure there are at least two distinct numbers to choose from
        feedback_label.value = "Please enter a value greater than 1."
        return

    # Save the current value of high_input before resetting (so we can restore it after restart)
    saved_high_value = high_input.value
    
    n1, n2 = respond(n)  # Generate the secret numbers
    numq = 0  # Reset numq (guesses count)

    # Update the score label immediately after game reset
    score_label.value = f"Score: {current_score}"
    
    feedback_label.value = "Game started! Make your guess."
    restart_checkbox.layout.display = 'none'  # Hide restart checkbox at the beginning
    restart_button.layout.display = 'none'  # Hide restart button at the beginning

    # Reset input boxes to default values
    low_input.value = 1
    high_input.value = saved_high_value  # Set high_input back to the saved value
    guess_low_input.value = 1
    guess_high_input.value = 10
    guess_number_input.value = 1

    # Display the guessing screen
    display_widgets()

# Function to check interval
def check_interval(end1, end2):
    global numq
    numq += 1  # Increment the number of guesses each time the player checks the interval

    # Check how many secret numbers are in the range
    count = test(n1, n2, end1, end2)
    feedback_label.value = f"There {'are' if count > 1 else 'is'} {count} secret number{'s' if count != 1 else ''} in your interval."
    feedback_label.value += f" (Guesses: {numq})"
    
    # Update the score label after each interval check
    score_label.value = f"Score: {current_score}"

# Function to make a guess for one secret number
def make_guess(guess_number):
    global current_score, numq
    numq += 1  # Increment the number of guesses each time the player submits a guess
    
    guessed_correctly = (guess_number in (n1, n2))

    # Update the score if the guess is correct
    if guessed_correctly:
        current_score = score(numq, max(n1, n2), current_score, guessed_correctly)
        display_result("YOU WON", "limegreen")  # Show win message
    else:
        # Display game over message
        display_result("GAME OVER", "orangered")  
    
    # Update the score label after each guess
    score_label.value = f"Score: {current_score}, Guesses: {numq}"



In this code chunk above, we used ChatGPT to create the functions that were formed by our logic and modified them to personalize it towards our specific needs. These functions are the foundation to how the game can be started, checking the interval that the player uses, and giving a response to a guess that is made. 

The run game function checks the starting interval that the player chooses and ensures that it is an interval that can be used. It then chooses two numbers from that interval to be the secret numbers using the function we created in our first code chunk. In addition, after the game is played, the function also allows the game to reset back to the default setting while also modifying the score to add whatever score they received. 

The next function simply checks the interval that the player uses as a guess in order to provide feedback on how many secret numbers are in the interval, as well as tallying the number of guesses. 

The final function in this code chunk checks the guess that the player makes and will decide the outcome of the game, depending on whether they were correct or not 

In [7]:
# Function to display the result with appropriate styling
def display_result(message, color):
    clear_output(wait=True)
    result_label.value = message  # Set the win/loss message in the label
    result_label.style = {'color': color, 'font-size': '36px', 'font-weight': 'bold', 'text-align': 'center'}
    
    # Show the restart checkbox
    restart_checkbox.layout.display = 'block'  # Show restart checkbox
    restart_checkbox.value = False  # Reset checkbox value

# Function to restart the game when the checkbox is checked
def check_restart(change):
    if change['new']:
        run_game()  # Call the game start function

# Display all widgets
def display_widgets():
    display(widgets.VBox([ 
        title_label, 
        low_input, high_input, start_button,
        guess_low_input, guess_high_input, check_button,
        score_label, feedback_label, guess_number_input, guess_button,
        result_label, restart_checkbox  # Include result label and restart checkbox
    ]))

In the code box above we made three functions in order to incorporate our game logic with display widgets. The display_result function creates a display for the win and loss message in the game interface and a restart checkbox. The check_restart function restarts the game when the check box is clicked in the game interface, which is why the run_game function is embedded in the function as well. The final function, display_widgets, displays all of the widgest such as title label, the low input and high input, start button, guess boxes, score, check box for restarting the game, and result label (feedback - won or lost). 

In [9]:
# Fun Title
title_label = widgets.Label(value="🎮 Guess the Secret Numbers! 🎮", 
                            style={'font-size': '30px', 'font-weight': 'bold', 'color': 'deepskyblue'}, 
                            layout=widgets.Layout(margin='10px 0 0 0'))

# Widgets with fun colors
low_input = widgets.IntText(value=1, description='Low:', style={'description_width': 'initial'}, layout=widgets.Layout(background_color='#E6F7FF', font_size='18px', width='250px'))
high_input = widgets.IntText(value=10, description='High:', style={'description_width': 'initial'}, layout=widgets.Layout(background_color='#E6F7FF', font_size='18px', width='250px'))
start_button = widgets.Button(description="Start Game", button_style='primary', layout=widgets.Layout(background_color='#90EE90', height='50px', width='300px', border='2px solid limegreen'))

# Inputs for checking interval
guess_low_input = widgets.IntText(value=1, description='Guess Low:', style={'description_width': 'initial'}, layout=widgets.Layout(background_color='#F0E68C', font_size='18px', width='250px'))
guess_high_input = widgets.IntText(value=10, description='Guess High:', style={'description_width': 'initial'}, layout=widgets.Layout(background_color='#F0E68C', font_size='18px', width='250px'))
check_button = widgets.Button(description="Check Interval", button_style='primary', layout=widgets.Layout(background_color='#FFDDC1', height='50px', width='300px', border='2px solid coral'))

# Input for guessing the secret number
guess_number_input = widgets.IntText(value=1, description='Guess the Number:', style={'description_width': 'initial'}, layout=widgets.Layout(background_color='#FFB6C1', font_size='18px', width='250px'))
guess_button = widgets.Button(description="Submit Guess", button_style='primary', layout=widgets.Layout(background_color='#FF6347', height='50px', width='300px', border='2px solid tomato'))

# Restart checkbox
restart_checkbox = widgets.Checkbox(value=False, description='Restart the game?', style={'description_width': 'initial'})
restart_checkbox.layout.display = 'none'  # Start hidden

# Result label for displaying win/loss messages
result_label = widgets.Label(value="", layout=widgets.Layout(margin='10px 0 0 0'))

# Labels
score_label = widgets.Label(value=f"Score: {current_score}", layout=widgets.Layout(margin='10px 0 0 0', background_color='#FFD700', font_size='20px'))
feedback_label = widgets.Label(value="", layout=widgets.Layout(margin='10px 0 0 0'))

# Button click event handlers
start_button.on_click(lambda b: run_game())
check_button.on_click(lambda b: check_interval(guess_low_input.value, guess_high_input.value))
guess_button.on_click(lambda b: make_guess(guess_number_input.value))
restart_checkbox.observe(check_restart, names='value')  # Observe checkbox value

# Display the widgets initially
display_widgets()

The code above is the widget design for the game. We used chatgpt to aid us in visualizing what we wanted our game to look like. This was done through various revisions through chatgpt for the game design. Since this was a first time using widgets, it helped ultimately helped us sythesize our ideas and incorporate all of our code in a user friendly GUI. 